In [1]:
#@title **Setup Environment**

import subprocess, time
print("Setting up environment...")
start_time = time.time()
all_process = [
    ['pip', 'install', 'torch==1.12.1+cu113', 'torchvision==0.13.1+cu113', '--extra-index-url', 'https://download.pytorch.org/whl/cu113'],
    ['pip', 'install', '-U', 'sentence-transformers'],
    ['pip', 'install', 'httpx'],
    ['pip', 'install', 'openai']
]
for process in all_process:
    running = subprocess.run(process,stdout=subprocess.PIPE).stdout.decode('utf-8')

end_time = time.time()
print(f"Environment set up in {end_time-start_time:.0f} seconds")

Setting up environment...
Environment set up in 167 seconds


In [2]:
!pip install openai


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os
import openai
import requests as r
from PIL import Image
import requests as r
from io import BytesIO  
OPENAI_API_KEY = ""

openai.api_key = OPENAI_API_KEY
openai.Model.list()

AuthenticationError: ignored

In [ ]:
#@markdown **Get personal access token in Mubert and define API methods**
email = "martyn.ben.ami@gmail.com" #@param {type:"string"}

r = httpx.post('https://api-b2b.mubert.com/v2/GetServiceAccess', 
    json={
        "method":"GetServiceAccess",
        "params": {
            "email": email,
            "license":"ttmmubertlicense#f0acYBenRcfeFpNT4wpYGaTQIyDI4mJGv5MfIhBFz97NXDwDNFHmMRsBSzmGsJwbTpP1A6i07AXcIeAHo5",
            "token":"4951f6428e83172a4f39de05d5b3ab10d58560b8",
            "mode": "loop"
        }
    })

rdata = json.loads(r.text)
assert rdata['status'] == 1, "probably incorrect e-mail"
pat = rdata['data']['pat']
print(f'Got token: {pat}')

In [ ]:
def query_image_creation_api(query: str, image_size:str = "256x256" ) -> Image:
    response = openai.Image.create(
    prompt=query,
    n=1,
    size=image_size
    )
    image_url = response['data'][0]['url']
    response = r.get(image_url, timeout=5)
    img = Image.open(BytesIO(response.content))
    return img

def query_text_completion_api(query: str, model_key = "text-davinci-003", num_tokens = 500, temp=0.7):
    template = f""" You are to convert a given topic_query to a list of prompts that are both descriptive and representative of the topic query
                    for the purpose of querying music and image generation services
                    This is the Query: {query}

    """


    resp = openai.Completion.create(
    model=model_key,
    prompt=template,
    max_tokens=num_tokens,
    temperature=temp
    )
    return resp

def query_bullet_point_api(query: str, model_key = "text-davinci-003", num_tokens = 500, num_bullet_point=2 temp=0.7) -> list:
    template = f""" You are to convert a given topic_query to a list of {num_bullet_point} bullet points that are both descriptive and represnetative of the topic query
     for the purpose of a slide show, they can be fun facts, interesting observations, or questions
        This is the Query: {query}

    """
    response = openai.Completion.create(
    prompt=query,
    max_tokens=num_tokens,
    temperature=temp,
    frequency_penalty=0,
    presence_penalty=0
    )
    bullet_points = response['choices'][0]['text'].replace("\n", "").split(".")
    return bullet_points[:num_of_bullet_points]

#create python code to create slides using the google slides api. each slide will be #created with a image file, a music file and a set of text.
#the image file will be an image from the OpenAI DALLE2 API.
#the music file will be a music file from the MBERT music creation api.
#the text will be a list of bullet points from the openAI GPT API.

#create a python code to create a video using the google slides api.
#the video will be created by taking the slides created by the previous python code and creating a video from them.
from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/presentations.readonly']

# The ID of a sample presentation.
PRESENTATION_ID = ''



#@title **Define Mubert methods and pre-compute things**

import numpy as np
from sentence_transformers import SentenceTransformer
minilm = SentenceTransformer('all-MiniLM-L6-v2')

mubert_tags_string = 'tribal,action,kids,neo-classic,run 130,pumped,jazz / funk,ethnic,dubtechno,reggae,acid jazz,liquidfunk,funk,witch house,tech house,underground,artists,mystical,disco,sensorium,r&b,agender,psychedelic trance / psytrance,peaceful,run 140,piano,run 160,setting,meditation,christmas,ambient,horror,cinematic,electro house,idm,bass,minimal,underscore,drums,glitchy,beautiful,technology,tribal house,country pop,jazz & funk,documentary,space,classical,valentines,chillstep,experimental,trap,new jack swing,drama,post-rock,tense,corporate,neutral,happy,analog,funky,spiritual,sberzvuk special,chill hop,dramatic,catchy,holidays,fitness 90,optimistic,orchestra,acid techno,energizing,romantic,minimal house,breaks,hyper pop,warm up,dreamy,dark,urban,microfunk,dub,nu disco,vogue,keys,hardcore,aggressive,indie,electro funk,beauty,relaxing,trance,pop,hiphop,soft,acoustic,chillrave / ethno-house,deep techno,angry,dance,fun,dubstep,tropical,latin pop,heroic,world music,inspirational,uplifting,atmosphere,art,epic,advertising,chillout,scary,spooky,slow ballad,saxophone,summer,erotic,jazzy,energy 100,kara mar,xmas,atmospheric,indie pop,hip-hop,yoga,reggaeton,lounge,travel,running,folk,chillrave & ethno-house,detective,darkambient,chill,fantasy,minimal techno,special,night,tropical house,downtempo,lullaby,meditative,upbeat,glitch hop,fitness,neurofunk,sexual,indie rock,future pop,jazz,cyberpunk,melancholic,happy hardcore,family / kids,synths,electric guitar,comedy,psychedelic trance & psytrance,edm,psychedelic rock,calm,zen,bells,podcast,melodic house,ethnic percussion,nature,heavy,bassline,indie dance,techno,drumnbass,synth pop,vaporwave,sad,8-bit,chillgressive,deep,orchestral,futuristic,hardtechno,nostalgic,big room,sci-fi,tutorial,joyful,pads,minimal 170,drill,ethnic 108,amusing,sleepy ambient,psychill,italo disco,lofi,house,acoustic guitar,bassline house,rock,k-pop,synthwave,deep house,electronica,gabber,nightlife,sport & fitness,road trip,celebration,electro,disco house,electronic'
mubert_tags = np.array(mubert_tags_string.split(','))
mubert_tags_embeddings = minilm.encode(mubert_tags)

from IPython.display import Audio, display
import httpx
import json

def get_track_by_tags(tags, pat, duration, maxit=20, autoplay=False, loop=False):
  if loop:
    mode = "loop"
  else:
    mode = "track"
  r = httpx.post('https://api-b2b.mubert.com/v2/RecordTrackTTM', 
      json={
          "method":"RecordTrackTTM",
          "params": {
              "pat": pat, 
              "duration": duration,
              "tags": tags,
              "mode": mode
          }
      })

  rdata = json.loads(r.text)
  assert rdata['status'] == 1, rdata['error']['text']
  trackurl = rdata['data']['tasks'][0]['download_link']

  print('Generating track ', end='')
  for i in range(maxit):
      r = httpx.get(trackurl)
      if r.status_code == 200:
          display(Audio(trackurl, autoplay=autoplay))
          break
      time.sleep(1)
      print('.', end='')

def find_similar(em, embeddings, method='cosine'):
    scores = []
    for ref in embeddings:
        if method == 'cosine': 
            scores.append(1 - np.dot(ref, em)/(np.linalg.norm(ref)*np.linalg.norm(em)))
        if method == 'norm': 
            scores.append(np.linalg.norm(ref - em))
    return np.array(scores), np.argsort(scores)

def get_tags_for_prompts(prompts, top_n=3, debug=False):
    prompts_embeddings = minilm.encode(prompts)
    ret = []
    for i, pe in enumerate(prompts_embeddings):
        scores, idxs = find_similar(pe, mubert_tags_embeddings)
        top_tags = mubert_tags[idxs[:top_n]]
        top_prob = 1 - scores[idxs[:top_n]]
        if debug:
            print(f"Prompt: {prompts[i]}\nTags: {', '.join(top_tags)}\nScores: {top_prob}\n\n\n")
        ret.append((prompts[i], list(top_tags)))
    return ret

def generate_track_by_prompt(prompt, duration, loop=False):
  _, tags = get_tags_for_prompts([prompt,])[0]
  try:
    get_track_by_tags(tags, pat, duration, autoplay=True, loop=loop)
  except Exception as e:
    print(str(e))



In [ ]:



def create_slides(queries):

    # Set up the Google Slides API
    slides_service = build('slides', 'v1', credentials=credentials)

    # Create a presentation
    presentation = slides_service.presentations().create(body={
        'title': 'My presentation'
    }).execute()
    presentation_id = presentation.get('presentationId')

    # Create a slide for each query
    for query in queries:
        image = query_image_creation_api(query)
        music_prompt = query
        bullet_points = query_text_creation_api(query, 5)
        pat, duration = generate_track_by_prompt(music_prompt, 60)
        # Add the slide
        requests = [
            {
                'createSlide': {
                    'objectId': query,
                    'slideLayoutReference': {
                        'predefinedLayout': 'TITLE_AND_TWO_COLUMNS'
                    }
                }
            },
            {
                'insertImage': {
                    'objectId': query,
                    'url': image,
                    'elementProperties': {
                        'pageObjectId': query,
                        'size': {
                            'height': {
                                'magnitude': 600,
                                'unit': 'PT'
                            },
                            'width': {
                                'magnitude': 1200,
                                'unit': 'PT'
                            }
                        },
                        'transform': {
                            'scaleX': 1,
                            'scaleY': 1,
                            'translateX': 0,
                            'translateY': 0,
                            'unit': 'PT'
                        }
                    }
                }
            },
            {
                'insertText': {
                    'objectId': query,
                    'text': '\n'.join(bullet_points),
                    'elementProperties': {
                        'pageObjectId': query,
                        'size': {
                            'height': {
                                'magnitude': 600,
                                'unit': 'PT'
                            },
                            'width': {
                                'magnitude': 600,
                                'unit': 'PT'
                            }
                        },
                        'transform': {
                            'scaleX': 1,
                            'scaleY': 1,
                            'translateX': 600,
                            'translateY': 0,
                            'unit': 'PT'
                        }
                    }
                }
            },
            {
                'insertAudio': {
                    'objectId': query,
                    'source': {
                        'url': pat
                    },
                    'elementProperties': {
                        'pageObjectId': query,
                        'size': {
                            'height': {
                                'magnitude': 200,
                                'unit': 'PT'
                            },
                            'width': {
                                'magnitude': 600,
                                'unit': 'PT'
                            }
                        },
                        'transform': {
                            'scaleX': 1,
                            'scaleY': 1,
                            'translateX': 600,
                            'translateY': 600,
                            'unit': 'PT'
                        }
                    }
                }
            }
        ]
        body = {
            'requests': requests
        }
        response = slides_service.presentations().batchUpdate(
            presentationId=presentation_id, body=body).execute()

    return presentation_id


def create_video(presentation_id):
    # Get the properties of the presentation
    presentation = slides_service.presentations().get(
        presentationId=presentation_id).execute()
    slides = presentation.get('slides')

    slides_ids = []
    for slide in slides:
        slides_ids.append(slide.get('objectId'))

    body = {
        'presentationId': presentation_id,
        'slideIds': slides_ids
    }

    video = slides_service.presentations().pages().getThumbnail(
        presentationId=presentation_id, body=body).execute()
    video_url = video.get('contentUrl')

    return video_url




In [ ]:
queries = ['query1', 'query2', 'query3']
presentation_id = create_slides(queries)
video_url = create_video(presentation_id)
print('Video URL:', video_url)